# Volumeter
To start up the system, run the cell below.

In [1]:
import import_ipynb
from backend.UI import *
start()

System started


Shutdown


AttributeError: module 'backend.Hardware' has no attribute 'mcSerial'

# TODO
- Kommenttikenttä
- Hyväksy mittaus
- Laske tilavuus